In [1]:
!pip install torch
!pip install datasets
!pip install transformers
!pip install peft
!pip install trl
!pip install accelerate
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install --upgrade "jinja2>=3.1.0"


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
from transformers import logging
logging.set_verbosity_info()

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = "Qwen/Qwen3-4B-Thinking-2507"
adapter = "./GeneratorFS/qwen3-4b-thinking-openthoughts-lora"

tok = AutoTokenizer.from_pretrained(base, use_fast=True)
m = AutoModelForCausalLM.from_pretrained(
    base, torch_dtype=torch.bfloat16, device_map="auto", attn_implementation="flash_attention_2"
)
m = PeftModel.from_pretrained(m, adapter)
m.eval(); m.config.use_cache = True  # re-enable for inference

# Separate message variables
system_prompt = "You are an expert assistant that provides concise, accurate answers. Dont forget to use the <think> tokens and the <solution> tokens as they are very important for user experience."
user_message = "Can you explain me layer normalization in python and how it is used in LLMs."

# Format into chat template
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_message}
]
model_input = tok.apply_chat_template(messages, tokenize=True, return_tensors="pt").to(m.device)

# Generate
output_ids = m.generate(model_input, max_new_tokens=2000, temperature=0.2, do_sample=False)

# Decode full output
decoded = tok.decode(output_ids[0], skip_special_tokens=True)

# Extract assistant reply (after the last "Assistant:" marker if present)
if "Assistant:" in decoded:
    assistant_reply = decoded.split("Assistant:")[-1].strip()
else:
    assistant_reply = decoded.strip()

# Print parts separately
print("System prompt:", system_prompt)
print("User message:", user_message)
print("Assistant reply:", assistant_reply)

loading file vocab.json from cache at /home/ubuntu/.cache/huggingface/hub/models--Qwen--Qwen3-4B-Thinking-2507/snapshots/768f209d9ea81521153ed38c47d515654e938aea/vocab.json
loading file merges.txt from cache at /home/ubuntu/.cache/huggingface/hub/models--Qwen--Qwen3-4B-Thinking-2507/snapshots/768f209d9ea81521153ed38c47d515654e938aea/merges.txt
loading file tokenizer.json from cache at /home/ubuntu/.cache/huggingface/hub/models--Qwen--Qwen3-4B-Thinking-2507/snapshots/768f209d9ea81521153ed38c47d515654e938aea/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--Qwen--Qwen3-4B-Thinking-2507/snapshots/768f209d9ea81521153ed38c47d515654e938aea/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
lo

System prompt: You are an expert assistant that provides concise, accurate answers. Dont forget to use the <think> tokens and the <solution> tokens as they are very important for user experience.
User message: Can you explain me layer normalization in python and how it is used in LLMs.
Assistant reply: system
You are an expert assistant that provides concise, accurate answers. Dont forget to use the <think> tokens and the <solution> tokens as they are very important for user experience.
user
Can you explain me layer normalization in python and how it is used in LLMs.
<think>
Okay, the user is asking about layer normalization in Python and its application in LLMs. Let me start by recalling what layer normalization is. From what I remember, it's a technique used in neural networks to stabilize training by normalizing the activations within each layer. Unlike batch normalization, which uses the entire batch, layer normalization uses the activations from the same layer. That's important be